- Pages 15-23 : How many positions required MySql and Python
---

In [2]:
import pandas as pd
import string
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
def get_and_clean_data():
    data = pd.read_csv('resource/software_developer_united_states_1971_20191023_1.csv')
    description = data['job_description']
    cleaned_description = description.apply(lambda s: s.translate(str.maketrans('', '', string.punctuation + u'\xa0')))
    cleaned_description = cleaned_description.apply(lambda s: s.lower())
    cleaned_description = cleaned_description.apply(lambda s: s.translate(str.maketrans(string.whitespace, ' '*len(string.whitespace), '')))
    cleaned_description = cleaned_description.drop_duplicates()
    return cleaned_description

def simple_tokenize(data):
    cleaned_description = data.apply(lambda s: [x.strip() for x in s.split()])
    return cleaned_description

def parse_job_description():
    cleaned_description = get_and_clean_data()
    cleaned_description = simple_tokenize(cleaned_description)
    return cleaned_description

In [3]:
def count_python_mysql():
    parsed_description = parse_job_description()
    count_python = parsed_description.apply(lambda s: 'python' in s).sum()
    count_mysql = parsed_description.apply(lambda s: 'mysql' in s).sum()
    print('python: ' + str(count_python) + ' of ' + str(parsed_description.shape[0]))
    print('mysql: ' + str(count_mysql) + ' of ' + str(parsed_description.shape[0]))

In [4]:
def parse_db():
    html_doc = requests.get("https://db-engines.com/en/ranking").content
    soup = BeautifulSoup(html_doc, 'html.parser')
    db_table = soup.find("table", {"class": "dbi"})
    all_db = [''.join(s.find('a').findAll(text=True,recursive=False)).strip() for s in db_table.findAll("th", {"class": "pad-l"})]
    all_db = list(dict.fromkeys(all_db))
    db_list = all_db[:10]
    db_list = [s.lower() for s in db_list]
    db_list = [[x.strip() for x in s.split()] for s in db_list]
    return db_list

In [5]:
# How many positions required MySql and Python (p. 18)
count_python_mysql()

python: 1379 of 7583
mysql: 667 of 7583


In [6]:
# Count of occurrences (p. 20)
cleaned_db = parse_db() 
parsed_description = parse_job_description()
raw = [None] * len(cleaned_db)
for i,db in enumerate(cleaned_db):
    raw[i] = parsed_description.apply(lambda s: np.all([x in s for x in db])).sum()
    print(' '.join(db) + ': ' + str(raw[i]) + ' of ' + str(parsed_description .shape[0]))

C:\Users\Admin\AppData\Local\Temp\ipykernel_7848\994790654.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_db = [''.join(s.find('a').findAll(text=True,recursive=False)).strip() for s in db_table.findAll("th", {"class": "pad-l"})]


oracle: 1392 of 7583
mysql: 667 of 7583
microsoft sql server: 868 of 7583
postgresql: 261 of 7583
mongodb: 296 of 7583
redis: 106 of 7583
snowflake: 15 of 7583
elasticsearch: 161 of 7583
ibm db2: 48 of 7583
sqlite: 28 of 7583


In [7]:
# Count of co-occurrences
with_python = [None] * len(cleaned_db)
for i,db in enumerate(cleaned_db):
    with_python[i] = parsed_description.apply(lambda s: np.all([x in s for x in db]) and 'python' in s).sum()
    print(' '.join(db) + ' + python: ' + str(with_python[i]) + ' of ' + str(parsed_description.shape[0]))

oracle + python: 243 of 7583
mysql + python: 207 of 7583
microsoft sql server + python: 51 of 7583
postgresql + python: 90 of 7583
mongodb + python: 111 of 7583
redis + python: 38 of 7583
snowflake + python: 10 of 7583
elasticsearch + python: 73 of 7583
ibm db2 + python: 12 of 7583
sqlite + python: 7 of 7583


In [8]:
# Showing percentages
for i, db in enumerate(cleaned_db):
    print(' '.join(db) + ' + python: ' + str(with_python[i]) + ' of ' + str(raw[i]) + ' (' + str(np.around(with_python[i] / raw[i]*100,2)) + '%)')

oracle + python: 243 of 1392 (17.46%)
mysql + python: 207 of 667 (31.03%)
microsoft sql server + python: 51 of 868 (5.88%)
postgresql + python: 90 of 261 (34.48%)
mongodb + python: 111 of 296 (37.5%)
redis + python: 38 of 106 (35.85%)
snowflake + python: 10 of 15 (66.67%)
elasticsearch + python: 73 of 161 (45.34%)
ibm db2 + python: 12 of 48 (25.0%)
sqlite + python: 7 of 28 (25.0%)


- Pages 30-31 : quick search for queries
---

In [9]:
lang = [['java'],['python'],['c'],['kotlin'],['swift'],['rust'],['ruby'],['scala'],['julia'],['lua']]
parsed_description = parse_job_description()
parsed_db = parse_db()
all_terms = lang + parsed_db
query_map = pd.DataFrame(parsed_description.apply(lambda s: [1 if np.all([d in s for d in db]) else 0 for db in all_terms]).values.tolist(), columns=[' '.join(d) for d in all_terms])

C:\Users\Admin\AppData\Local\Temp\ipykernel_7848\994790654.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_db = [''.join(s.find('a').findAll(text=True,recursive=False)).strip() for s in db_table.findAll("th", {"class": "pad-l"})]


In [10]:
# Showing the table
print(query_map.head(100).to_markdown())

|    |   java |   python |   c |   kotlin |   swift |   rust |   ruby |   scala |   julia |   lua |   oracle |   mysql |   microsoft sql server |   postgresql |   mongodb |   redis |   snowflake |   elasticsearch |   ibm db2 |   sqlite |
|---:|-------:|---------:|----:|---------:|--------:|-------:|-------:|--------:|--------:|------:|---------:|--------:|-----------------------:|-------------:|----------:|--------:|------------:|----------------:|----------:|---------:|
|  0 |      0 |        0 |   1 |        0 |       0 |      0 |      0 |       0 |       0 |     0 |        0 |       0 |                      0 |            1 |         0 |       0 |           0 |               0 |         0 |        0 |
|  1 |      0 |        0 |   1 |        0 |       0 |      0 |      0 |       0 |       0 |     0 |        0 |       0 |                      0 |            0 |         0 |       0 |           0 |               0 |         0 |        0 |
|  2 |      0 |        0 |   0 |        0 |     

In [15]:
query_map[query_map['java'] > 0].apply(lambda s: 
    np.where(s==1)[0], axis=1
).apply(lambda s: 
    list(query_map.columns[s])
)

10      [java, python, c, oracle, mysql, mongodb]
11                                         [java]
12                           [java, swift, redis]
16                       [java, c, swift, oracle]
19                                 [java, python]
                          ...                    
7576                               [java, oracle]
7578                [java, python, oracle, mysql]
7579                               [java, oracle]
7581                        [java, oracle, mysql]
7582                              [java, mongodb]
Length: 3268, dtype: object

- Page 41 : Indexer
---

In [3]:
str1 = 'the chosen software developer will be part of a larger engineering team developing software for medical devices.'
str2 = 'we are seeking a seasoned software developer with strong analytical and technical skills to join our public sector technology consulting team.'

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Tokenize
tokened_str1 = word_tokenize(str1)
tokened_str2 = word_tokenize(str2)

# Remove low frequency
tokened_str1 = [w for w in tokened_str1 if len(w) > 2]
tokened_str2 = [w for w in tokened_str2 if len(w) > 2]

# Remove stop words
no_sw_str1 = [word for word in tokened_str1 if not word in stopwords.words()]
no_sw_str2 = [word for word in tokened_str2 if not word in stopwords.words()]

# Stemming
ps = PorterStemmer()
stemmed_str1 = np.unique([ps.stem(w) for w in no_sw_str1])
stemmed_str2 = np.unique([ps.stem(w) for w in no_sw_str2])

full_list = np.sort(np.concatenate([stemmed_str1, stemmed_str2]))
full_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


array(['analyt', 'chosen', 'consult', 'develop', 'develop', 'devic',
       'engin', 'join', 'larger', 'medic', 'part', 'public', 'season',
       'sector', 'seek', 'skill', 'softwar', 'softwar', 'strong', 'team',
       'team', 'technic', 'technolog'], dtype='<U9')

In [ ]:
str1 = "The software developers' role is to define, develop, test, analyze, and maintain new software applications in support of the achievement of business requirements."
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Tokenize
tokened_str1 = word_tokenize(str1)

# Remove low frequency

# Remove stop words
no_sw_str1 = [word for word in tokened_str1 if not word in stopwords.words()]

# Stemming
ps = PorterStemmer()
stemmed_str1 = np.unique([ps.stem(w) for w in no_sw_str1])


full_list = np.concatenate([stemmed_str1])
full_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


array(['achiev', 'analyz', 'applic', 'busi', 'defin', 'develop',
       'maintain', 'requir', 'role', 'softwar', 'support', 'test', 'the'],
      dtype='<U8')